# Защита персональных данных клиентов  
Необходимо предложить механизм преобразования данных пользователей, который бы помог защитить личную информацию, но не отразился бы на предсказании модели линейной регрессии

[1. Загрузка данных](#data_download)  
[2. Предобработка данных](#data_preprocessing)   
[3. Предложение метода защиты данных](#data_safety_method)    
[4. Доказательство корректности метода](#proof)  
[5. Обучение и проверка качества линейной регрессии до применения метода](#before)  
[6. Обучение и проверка качества линейной регрессии после применения метода](#after)  
[7. Выводы](#conclusion)

<a id='data_download'></a>
### 1. Загрузка данных 

<a id='data_preprocessing'></a>
### 2. Предобработка данных

<a id='data_safety_method'></a>
### 3. Предложение метода защиты данных

 <a id='proof'></a>
### 4. Доказательство корректности метода

Необходимые формулы:  
  
$ (AB)^{T} = B^{T}A^{T} $  
$ (AB)^{-1} = B^{-1}A^{-1} $  
$ AA^{-1} = E = A^{-1}A $


Пусть $ X^{'} $ - матрица признаков $ m*n $  

Пусть $ W_{1} $ - вектор весов  $(1*m)$ , $W_{0}$  - скаляр

Тогда предсказание линейной регрессии можно записать:
$$Y = X^{'} *W_{1} + W_{0} $$
для упрощения расчетов добавим к матрице $ X^{'} $ единичный столбец, а скаляр $ W_{0}$ добавим в вектор $ W_{1} $

$$\begin{bmatrix} 1 & x_{12} & x_{13} & ... & x_{1n} \\ 1 & x_{21} & x_{22} & ... & x_{2n} \\ ...\\ 1 & x_{m2} & x_{m3} & ... & x_{mn}\end{bmatrix} *  \begin{bmatrix} w_{0} \\ w_{1} \\ ... \\ w_{n}\end{bmatrix}$$ 

Обозначим новую матрицу признаков с еденичным столбцом за $ X $ , а вектор весов - $ W $

**Формула предсказания линейной регрессии:**
$ Y = X*W $ 


Если принять метрику качества - минимальное значение **MSE** -  среднеквадратичноей ошибки, то для поиска минимального решения необходимо минимизировать:  

$ (Y - XW)^{T}(Y-XW) -> min $  


оптимальное решение относительно W находится приравниваем к нулю дифференциала этой функции по W   
 
В результате преобразований получим  

$ W = (X^{T}X)^{-1}X^{T}Y $

### Докажем, что умножение матрицы X на обратимую матрицу не ухудшит качество предсказаний

### Умножим матрицу признаков $ X $ на обратимую матрицу $ Q $ размерности (n*m) 

$ W^{'} = ((XQ)^{T}XQ)^{-1}(XQ)^{T}Y $  

$ W^{'} = (Q^{T}X^{T}XQ)^{-1}Q^{T}X^{T}Y $

$ W^{'} = Q^{-1}(X^{T}X)^{-1}(Q^{T})^{-1}Q^{T}X^{T}Y $

$ (Q^{T})^{-1}Q^{T}  = E $  
 
отсюда:

$ W^{'} = Q^{-1}(X^{T}X)^{-1}X^{T}Y $ , где  
$ (X^{T}X)^{-1}X^{T}Y $ равно $ W $

Получаем,что   

$ W^{'} = Q^{-1}*W $

Подставим $ Q^{-1}W $ вместо веса $ W^{'} $  в предсказание модели $ Y^{'} = XQW^{'} $ 

$ Y^{'} = XQ*Q^{-1}W $  
$ Y^{'} = XEW $  
$ Y^{'} = XW = Y $  

Доказали, что от умножения на обратимую матрицу, предсказания модели **не меняются**. Меняются только веса, те модель необходимо переобучить.

<a id='before'></a>
### 5. Обучение и проверка качества линейной регрессии до применения метода

<a id='after'></a>
### 6. Обучение и проверка качества линейной регрессии после применения метода

<a id='conclusion'></a>
### 7. Выводы